In [4]:
import logging

import numpy as np
import torch
import tqdm
from torch import nn
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [5]:
import pickle

In [4]:
import pickle
with open('good_list.pkl', 'wb') as f:
    pickle.dump(good_list, f)
with open('orderprob.pkl', 'wb') as f:
    pickle.dump(df.problem, f)
        

NameError: name 'good_list' is not defined

In [6]:
df=pd.read_csv('attempts.csv')

the problem 25 is never given to a student so we can replace the name of problem 86 with 25

In [7]:
df.problem=df.problem.replace(86,25)

In [8]:
df

,id,student,problem,start,solved,time,n_edits,n_executions,program,problemset
0,4,29,2,2017-11-10 12:20:05.152265+00:00,True,24,3,2,lrf,2
1,5,29,12,2017-11-10 12:20:32.155447+00:00,True,19,4,1,rrff,5
2,6,29,10,2017-11-10 12:20:54.080259+00:00,True,10,3,1,sff,4
3,7,29,14,2017-11-10 12:21:06.966330+00:00,True,18,3,1,frl,3
4,8,29,11,2017-11-10 12:21:33.061485+00:00,True,27,4,1,R4{fs},7
...,...,...,...,...,...,...,...,...,...,...
285806,308258,32173,50,2020-10-01 17:22:44.928241+00:00,False,5689,9,2,R4{l}R2{r},13
285807,308259,32252,51,2020-10-01 17:24:41.451870+00:00,True,42,5,3,fff,1
285808,308260,32252,44,2020-10-01 17:25:27.317684+00:00,True,16,3,2,lff,2
285809,308261,32252,14,2020-10-01 17:25:47.455266+00:00,True,74,18,4,frl,3


We do a classic logistic regression to find the IRT parameters to rename the problems by their difficulty ranking

In [9]:
estimators = [
    ('onehot', OneHotEncoder()),
    ('lr', LogisticRegression())
]
pipe = Pipeline(estimators)

# Just check the encoded variables
ohe = OneHotEncoder()
print(ohe.fit_transform(df[['student', 'problem']]))
pipe.fit(df[['student', 'problem']], df['solved'])
print(pipe.predict_proba(df[['student', 'problem']]))
cofs=pipe[1].coef_
diffik=pd.Series(index=np.sort(df.problem.unique()),data=cofs.reshape(-1,)[-85:])

#probably not optimal way to change the value of the exercice
scv=diffik.sort_values(ascending=False).index.values
po=[]
for j in df.problem.values:
    po.append((np.where(scv==j)[0]+1)[0])
df.problem=po    

  (0, 1)	1.0
  (0, 20557)	1.0
  (1, 1)	1.0
  (1, 20567)	1.0
  (2, 1)	1.0
  (2, 20565)	1.0
  (3, 1)	1.0
  (3, 20569)	1.0
  (4, 1)	1.0
  (4, 20566)	1.0
  (5, 1)	1.0
  (5, 20556)	1.0
  (6, 1)	1.0
  (6, 20558)	1.0
  (7, 1)	1.0
  (7, 20604)	1.0
  (8, 1)	1.0
  (8, 20572)	1.0
  (9, 1)	1.0
  (9, 20563)	1.0
  (10, 2)	1.0
  (10, 20557)	1.0
  (11, 2)	1.0
  (11, 20586)	1.0
  (12, 2)	1.0
  :	:
  (285798, 20580)	1.0
  (285799, 20554)	1.0
  (285799, 20569)	1.0
  (285800, 20554)	1.0
  (285800, 20581)	1.0
  (285801, 20554)	1.0
  (285801, 20563)	1.0
  (285802, 20554)	1.0
  (285802, 20565)	1.0
  (285803, 20554)	1.0
  (285803, 20567)	1.0
  (285804, 20554)	1.0
  (285804, 20574)	1.0
  (285805, 20554)	1.0
  (285805, 20575)	1.0
  (285806, 20495)	1.0
  (285806, 20605)	1.0
  (285807, 20555)	1.0
  (285807, 20606)	1.0
  (285808, 20555)	1.0
  (285808, 20599)	1.0
  (285809, 20555)	1.0
  (285809, 20569)	1.0
  (285810, 20555)	1.0
  (285810, 20621)	1.0


/opt/homebrew/anaconda3/envs/newenv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[[0.01447939 0.98552061]
 [0.0060792  0.9939208 ]
 [0.00706871 0.99293129]
 ...
 [0.03005869 0.96994131]
 [0.04542564 0.95457436]
 [0.12874895 0.87125105]]


a way to keep the difficulty of the exercices for the futur reward

In [8]:
with open('diffikrobo.pkl', 'wb') as f:
    pickle.dump(pd.Series(index=np.arange(1,86),data=diffik.sort_values(ascending=False).values), f)


We encode the vector for DKT for each student

In [10]:
#start by doing a dataframe of the history success for each student 
ohe=OneHotEncoder()
l=ohe.fit_transform(df[['problem','solved']])
  

In [11]:
stud_list={}
for j in df.student.unique():
    stud_list[j]=df[df.student==j][['problem','solved']]

In [12]:
encoded_l={}
for j in df.student.unique():
    encoded_l[j]=l[stud_list[j].index]

In [13]:

good_list={}
for j in df.student.unique():
    m=encoded_l[j].toarray()
    good_list[j]=torch.zeros(len(m),170)
    for i in range(len(m)):
        if m[i,-2]:
            good_list[j][i][:85]=torch.Tensor(m[i,:-2])
        else:
            good_list[j][i][-85:]=torch.Tensor(m[i,:-2])
            
            

Splitting the training set and test set

In [14]:

mm=np.split(df.student.unique(),3)
np.r_[mm[0],mm[1]]
train_l={good_list[j] for j in np.r_[mm[0],mm[1]] }


In [15]:
test_l={good_list[j] for j in mm[2]}

Code for the LSTM architecture 

In [16]:
class Net(nn.Module):
    def __init__(self, num_questions, hidden_size, num_layers):
        super(Net, self).__init__()
        self.hidden_dim = hidden_size
        self.layer_dim = num_layers
        self.lstm = nn.LSTM(num_questions * 2, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(self.hidden_dim, num_questions)

    def forward(self, x):
        h0 = Variable(torch.zeros(self.layer_dim,  self.hidden_dim))
        c0=  Variable(torch.zeros(self.layer_dim,  self.hidden_dim))
        out, (a,b)= self.lstm(x, (h0,c0))
        res = torch.sigmoid(self.fc(out))
        return res


In [62]:

def process_raw_pred(raw_question_matrix, raw_pred, num_questions: int) -> tuple:
    questions = torch.nonzero(raw_question_matrix)[0] % num_questions
    
    pred = raw_pred[questions]
  
    truth = (torch.nonzero(raw_question_matrix)[0] // num_questions)*(torch.nonzero(raw_question_matrix)[0]>84)
    return pred, truth



In [ ]:
net(torch.zeros(1,170))

In [63]:

class DKT():
    def __init__(self, num_questions, hidden_size, num_layers):
        super(DKT, self).__init__()
        self.num_questions = num_questions
        self.dkt_model = Net(num_questions, hidden_size, num_layers)

    def train(self, train_data, test_data=None, *, epoch: int, lr=0.002) -> ...:
        loss_function = nn.BCELoss()
        optimizer = torch.optim.Adam(self.dkt_model.parameters(), lr)

        for e in range(epoch):
            all_pred, all_target = torch.Tensor([]), torch.Tensor([])
            for batch in tqdm.tqdm(train_data, "Epoch %s" % e):
                integrated_pred = self.dkt_model(batch)
                batch_size = batch.shape[0]
                for student in range(batch_size):
                    pred, truth = process_raw_pred(batch[student], integrated_pred[student], self.num_questions)
                    all_pred = torch.cat([all_pred, pred])
                    all_target = torch.cat([all_target, truth.float()])

            loss = loss_function(all_pred, all_target)
            # back propagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            print("[Epoch %d] LogisticLoss: %.6f" % (e, loss))

            if test_data is not None:
                auc = self.eval(test_data)
                print("[Epoch %d] auc: %.6f" % (e, auc))

    def eval(self, test_data) -> float:
        self.dkt_model.eval()
        y_pred = torch.Tensor([])
        y_truth = torch.Tensor([])
        for batch in tqdm.tqdm(test_data, "evaluating"):
            integrated_pred = self.dkt_model(batch)
            batch_size = batch.shape[0]
            for student in range(batch_size):
                pred, truth = process_raw_pred(batch[student], integrated_pred[student], self.num_questions)
                y_pred = torch.cat([y_pred, pred])
                y_truth = torch.cat([y_truth, truth])

        return roc_auc_score(y_truth.detach().numpy(), y_pred.detach().numpy())

In [58]:
    def save(self, filepath):
        torch.save(self.dkt_model.state_dict(), filepath)
        logging.info("save parameters to %s" % filepath)

    def load(self, filepath):
        self.dkt_model.load_state_dict(torch.load(filepath))
        logging.info("load parameters from %s" % filepath)

In [64]:
dkt=DKT(num_questions=85,hidden_size=5,num_layers=3)

In [65]:
dkt.train(train_l,epoch=50,lr=0.02,test_data=test_l)

Epoch 0:   0%|          | 0/13704 [00:00<?, ?it/s]

/var/folders/rh/v2m3z36j1j71mqlvn91zps5c0000gn/T/ipykernel_22841/543832646.py:6: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  truth = (torch.nonzero(raw_question_matrix)[0] // num_questions)*(torch.nonzero(raw_question_matrix)[0]>84)
Epoch 0: 100%|██████████| 13704/13704 [01:11<00:00, 192.66it/s]


[Epoch 0] LogisticLoss: 0.714090


evaluating: 100%|██████████| 6852/6852 [00:58<00:00, 116.24it/s]

In [24]:
dkt.eval(test_l)

saving the model weights

In [23]:
torch.save(dkt.dkt_model.state_dict(),'/Users/samuelgirard/zr-obp/weightslstm.pth')

In [26]:
test_l